In [1]:
pip install langchain langchain_community langchain_groq unstructured faiss-cpu python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.6 MB/s eta 0:00:00
   ━━

In [2]:
pip install langchain-huggingface

In [3]:
pip install azure-ai-inference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 14.3 MB/s eta 0:00:00


In [6]:
import os
import time
from google.colab import userdata
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
load_dotenv()
os.environ['GROQ_API_KEY'] = "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"



def process_urls(urls):
    loader = UnstructuredURLLoader(urls=urls)
    print("data loading.....")
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000
    )
    print("splitting text....")
    docs = text_splitter.split_documents(data)

    embedding = HuggingFaceEmbeddings()

    vectorstore = FAISS.from_documents(docs, embedding)
    print("indexing.....")
    time.sleep(2)
    return vectorstore


def querying_index(query,vectorstore):
    llm = ChatGroq(temperature=0.9)
    chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())
    result = chain.invoke({"question":query})
    return result

def main():
    urls = ["https://und.edu/"]
    process_urls_flag = input("yes/no").strip().lower()
    if process_urls_flag=="yes":
        vectorstore = process_urls(urls)
        print("Processing completed. FAISS index created.")
    while(True):
      query = input("question:").strip()
      if query:
          result = querying_index(query,vectorstore)
          if result:
              print("\nAnswer:")
              print(result["answer"])

if __name__ == "__main__":
    main()





yes/noyes
data loading.....
splitting text....


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2149 > 1024). Running this sequence through the model will result in indexing errors



Answer:
The PDF is about the University of North Dakota (UND), highlighting various programs, student experiences, and the university's commitment to diversity, inclusion, and providing a sense of belonging for all students. Specific programs mentioned include Data Science, Aerospace & Aviation, Arts & Science, Business, Education, Engineering, Medicine, Nursing & Health, and Social Work.




KeyboardInterrupt: Interrupted by user